In [23]:
#Mutation based fuzzer


import random
import string

#config
MAX_NUMBER_OF_MUTATORS = 10
INSERT_MAX_LENGTH = 10
DELETE_MAX_LENGTH = 10

ALL_ASCII = []
for i in range(0, 128):
    ALL_ASCII.append(chr(i))

def mutator_insert(input_data):
    length = random.randrange(0, INSERT_MAX_LENGTH)
    position = random.randrange(0, len(input_data))
    random_bytes = bytearray()
    #random_bytes = ""
    for i in range(0, length):
        random_bytes += bytes([random.randrange(0, 255)])
        #random_bytes += random.choice(string.printable)
    return input_data[:position] + random_bytes + input_data[position:]

def mutator_delete(input_data):
    length = len(input_data)
    position = random.randrange(8, length)
    count = random.randrange(0, min(length, DELETE_MAX_LENGTH))
    return input_data[:position] + input_data[position+count:]
        
#active_mutators=[mutator_insert, mutator_delete]
active_mutators=[mutator_insert]

def mutate_corpus(corpus,run_id):
    random.seed(run_id)
    active_corpus=random.choice(corpus)
    return mutate(active_corpus, run_id)

def mutate(input_data, run_id):
    random.seed(run_id)
    number_of_mutations = random.randint(0, MAX_NUMBER_OF_MUTATORS)
    for i in range(0, number_of_mutations):
        mutator = random.choice(active_mutators)
        input_data = mutator(input_data)
    return input_data

    

In [24]:
mutate(bytes(10), 1)
import glob
print(glob.glob("./corpus/*"))
corpus_files = glob.glob("./corpus/*")
corpus=[]

for file in corpus_files:
    with open(file, "rb") as f:
        corpus.append(f.read())

        
print(mutate_corpus(corpus,1))

['./corpus/test1', './corpus/test2', './corpus/test3', './corpus/test4', './corpus/test5']
b'{"menu": {\n  "id": "file|\x07\xe4",\n  "~\xc2sxvalue": "File",\n  "popup": {\n    "menuitem": [\n      {"value": "New", "onclick": "CreateNewDoc()"},\n      {"value": "Open", "onclick": "OpenDoc()"},\n      {"value": "Close", "onclick": "CloseDoc()"}\n    ]\n  }\n}}\n'


In [25]:
import subprocess

def runner(testcase):
    with open("testcase.json", "wb") as f:
        f.write(testcase)
        f.close()
    result = subprocess.run(["./cJSON-fuzzer", "testcase.json"], shell=False, timeout=1)

#def runner(testcase):
#    result = subprocess.run(["./coverage", testcase], shell=False, timeout =1)
#    return result.returncode

In [26]:
runner(b"asdf")

#Fuzzing

In [27]:
#!rm coverage.gcda 
#!rm coverage.gcno
#!clang -fsanitize=address cJSON.c cJSON-fuzzer.c -o cJSON-fuzzer -lm
!gcc --coverage cJSON.c cJSON-fuzzer.c -o cJSON-fuzzer -lm
#!gcc --coverage coverage.c -o coverage


#Dry run to determine base coverage
for c in corpus:
    runner(c)

def get_coverage():
    r = subprocess.run(["gcov", "cJSON"], stdout=subprocess.PIPE)
    return int(float(r.stdout.decode('utf-8').split('Lines executed:')[1].split('% of')[0]))
    
run_id =0 
current_coverage=get_coverage()
print(f"Base Coverage is {current_coverage}%")
while True:
    testcase = mutate_corpus(corpus, run_id)
    result=runner(testcase)

    coverage =get_coverage()
    
    #check if coverage increased
    if(coverage > current_coverage):
        current_coverage = coverage
        corpus.append(testcase)
        print("Current coverage - " + str(current_coverage) + "%")
    
    #if result != 0:
        #print(f"Success : {run_id}")
        
    run_id +=1

cJSON-fuzzer.c: In function ‘main’:
cJSON-fuzzer.c:22:22: warning: implicit declaration of function ‘malloc’ [-Wimplicit-function-declaration]
   22 |  char *file_buffer = malloc(file_length+1);
      |                      ^~~~~~
cJSON-fuzzer.c:22:22: warning: incompatible implicit declaration of built-in function ‘malloc’
cJSON-fuzzer.c:4:1: note: include ‘<stdlib.h>’ or provide a declaration of ‘malloc’
    3 | #include "cJSON.h"
  +++ |+#include <stdlib.h>
    4 | 
Base Coverage is 13%
Current coverage - 15%
Current coverage - 16%
Current coverage - 17%
Current coverage - 18%
Current coverage - 21%


TimeoutExpired: Command '['./cJSON-fuzzer', 'testcase.json']' timed out after 0.9999234679999063 seconds

In [ ]:
#testcase = mutate_corpus(corpus, 1931)
#result=runner(testcase)
#print(testcase)
#print(result)